Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
smx=SGDClassifier()
lr=LogisticRegression(random_state = 0)
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
dt= DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=2, min_samples_leaf=5)
km=KMeans(n_clusters=2)
rf = RandomForestClassifier(n_estimators=100, criterion = 'entropy', random_state = 0)
sv=SVC(kernel='poly')
xgb=XGBClassifier()
gb=GradientBoostingClassifier()
gnb=GaussianNB()

Py-Torch,sentence embeddings


In [ ]:
!pip install pytorch-pretrained-bert
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "Here is the sentence I want embeddings for."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)
# Define a new example sentence with multiple meanings of the word "bank"
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))
    # Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
print ("Number of layers:", len(encoded_layers))
layer_i = 0

print ("Number of batches:", len(encoded_layers[layer_i]))
batch_i = 0

print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(encoded_layers, dim=0)

token_embeddings.size()
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

['[CLS]', 'here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.', '[SEP]']
[CLS]           101
after         2,044
stealing     11,065
money         2,769
from          2,013
the           1,996
bank          2,924
vault        11,632
,             1,010
the           1,996
bank          2,924
robber       27,307
was           2,001
seen          2,464
fishing       5,645
on            2,006
the           1,996
mississippi   5,900
river         2,314
bank          2,924
.             1,012
[SEP]           102
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Number of layers: 12
Number of batches: 1
Number of tokens: 22
Number of hidden units: 768
Shape is: 22 x 3072


In [ ]:
import torch
torch.Size([12, 1, 22, 768])
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([22, 12, 768])

Loading the dataset

In [ ]:
!pip install pytreebank


import pytreebank
sst = pytreebank.load_sst()

In [ ]:
import pandas as pd
data_train = [
                (
                    tree.to_lines()[0],
                    tree.label
                )
                for tree in sst['train']
            ]
data_test = [(
  tree.to_lines()[0],
  tree.label
)
for tree in sst['test']
]
x_train=pd.DataFrame(data_train,columns=['sentence','label'])
x_test=pd.DataFrame(data_test,columns=['sentence','label'])
x_train_sent=x_train[['sentence']]
print(x_train_sent)
y_train_ind=x_train['label']
y_train_ind
x_test_sent=x_test[['sentence']]
y_test_ind=x_test['label']
x_test_sent


                                               sentence
0     The Rock is destined to be the 21st Century 's...
1     The gorgeously elaborate continuation of `` Th...
2     Singer/composer Bryan Adams contributes a slew...
3     You 'd think by now America would have had eno...
4                  Yet the act is still charming here .
...                                                 ...
8539                                    A real snooze .
8540                                     No surprises .
8541  We 've seen the hippie-turned-yuppie plot befo...
8542  Her fans walked out muttering words like `` ho...
8543                                In this case zero .

[8544 rows x 1 columns]


,sentence
0,Effective but too-tepid biopic
1,If you sometimes like to go to the movies to h...
2,"Emerges as something rare , an issue movie tha..."
3,The film provides some great insight into the ...
4,Offers that rare combination of entertainment ...
...,...
2205,An imaginative comedy/thriller .
2206,"( A ) rare , beautiful film ."
2207,( An ) hilarious romantic comedy .
2208,Never ( sinks ) into exploitation .


Pre-Processing

In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=2500, lower=True,split=' ')
tokenizer.fit_on_texts(x_train_sent['sentence'])
tokenizer.fit_on_texts(x_test_sent['sentence'])
# print(len(tokenizer.word_index))
X = tokenizer.texts_to_sequences(list(x_train_sent['sentence']))
x=tokenizer.texts_to_sequences(list(x_test_sent['sentence']))
x=pad_sequences(x)
X = pad_sequences(X)
print(X.size)

393024


Vocabulary 

In [ ]:
import pickle
# embeding_matrix=np.zeros((len(),300))
vocabulary=[]
for token,index in tokenizer.word_index.items():
  vocabulary.insert(index,token) 
#pickle.dump(vocabulary,open('/content/drive/MyDrive/vocabulary.pickle', 'wb'))
# for wor in vocabulary:
# len(vocabulary)
# len(tokenizer.word_index.keys())  

In [ ]:
import pickle
vocabulary=pickle.load(open('/content/drive/MyDrive/vocabulary.pickle', 'rb'))
glove_embed_sst=pickle.load(open('/content/drive/MyDrive/Glove_embed_sst.pkl', 'rb'))

Embedding matrix

In [ ]:
embedding_matrix=np.zeros((len(vocabulary),300))
for word,vec in glove_embed_sst.items():
  ind=vocabulary.index(word)
  embedding_matrix[ind]=vec
embedding_matrix.shape


(17087, 300)

LSTM and Bi-LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding,Bidirectional
from keras.layers import Dense
lstm_out = 300
batch_size = 32
model = Sequential()
model.add(Embedding(len(vocabulary), 300, trainable = False, weights=[embedding_matrix]))
#model.add(Embedding(len(vocabulary), 300,input_length = X.shape[1]))
model.add(Dropout(0.2))
model.add(Masking(mask_value=0))
#model.add(Bidirectional(LSTM(lstm_out, recurrent_dropout = 0.2, dropout = 0.2)))
model.add(LSTM(lstm_out, recurrent_dropout = 0.2, dropout = 0.2))
# model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 300)         5126100   
_________________________________________________________________
dropout (Dropout)            (None, None, 300)         0         
_________________________________________________________________
masking (Masking)            (None, None, 300)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 300)               721200    
_________________________________________________________________
dense (Dense)                (None, 5)                 1505      
Total params: 5,848,805
Trainable params: 722,705
Non-trainable params: 5,126,100
_________________________________________________________________
None


In [ ]:
ohe = OneHotEncoder()
Y = ohe.fit_transform(np.array(y_train_ind).reshape(-1,1)).toarray()
y = ohe.fit_transform(np.array(y_test_ind).reshape(-1,1)).toarray()
model.fit(X,Y, batch_size =batch_size, epochs = 5)

Epoch 1/5
267/267 [==============================] - 162s 507ms/step - loss: 1.5874 - accuracy: 0.2604
Epoch 2/5
267/267 [==============================] - 136s 509ms/step - loss: 1.5175 - accuracy: 0.3118
Epoch 3/5
267/267 [==============================] - 133s 497ms/step - loss: 1.4758 - accuracy: 0.3495
Epoch 4/5
267/267 [==============================] - 135s 507ms/step - loss: 1.4307 - accuracy: 0.3838
Epoch 5/5
267/267 [==============================] - 135s 506ms/step - loss: 1.3747 - accuracy: 0.4125


In [ ]:
 score,evalu=model.evaluate(x,y,verbose=2,batch_size=batch_size)

70/70 - 7s - loss: 1.4954 - accuracy: 0.3430


In [ ]:
 print(evalu)

0.34298643469810486


BERT embeddings

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 92kB 4.2MB/s 
     |████████████████████████████████| 2.5MB 8.0MB/s 
     |████████████████████████████████| 1.2MB 26.9MB/s 
     |████████████████████████████████| 3.3MB 45.1MB/s 
     |████████████████████████████████| 901kB 37.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=531e87b845421084ef19761905f75d8348e99140d2c10a7dde9c8698ed92dbf6
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
#sentences = ['This framework generates embeddings for each input sentence',  'Sentences are passed as a list of string.', 'The quick brown fox jumps over the lazy dog.']
# sentence_embeddings = model.encode(x_train_sent['sentence'])

In [ ]:
sentence_embeddings_test=model.encode(x_test_sent['sentence'])

Sentence Embeddings of train and test data

In [ ]:
# pickle.dump(sentence_embeddings_test,open('/content/drive/My Drive/embeding_test.pickle', 'wb'))
# pickle.dump(sentence_embeddings,open('/content/drive/My Drive/embeding.pickle', 'wb'))
# !cat /content/drive/My\ Drive/f.txt

In [ ]:
import pickle
sentence_embeddings=pickle.load(open('/content/drive/MyDrive/embeding.pickle', 'rb'))
len(sentence_embeddings)
sentence_embeddings_test=pickle.load(open('/content/drive/MyDrive/embeding_test.pickle','rb'))
#print(sentence_embeddings_test[0].size)
#print(x_train_sent)

Neural Network

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
# sc = StandardScaler()
# X = sc.fit_transform(X)
ohe = OneHotEncoder()
y_test_ind = ohe.fit_transform(y_sst2_tes.reshape(-1,1)).toarray()
y_train_ind = ohe.fit_transform(y_sst2_t.reshape(-1,1)).toarray()
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)
model = Sequential()
model.add(Dense(128, input_dim=(768), activation='relu'))
# model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
np.array(sentence_embeddings_test).shape

(2210, 768)

In [ ]:
history = model.fit(np.array(sentence_embeddings), y_train_ind, epochs=10, batch_size=32)
y_pred = model.predict(np.array(sentence_embeddings_test))
y_pred=np.array(y_pred)
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
test = list()
for i in range(len(y_test_ind)):
    test.append(np.argmax(y_test_ind[i]))
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

Epoch 1/10
267/267 [==============================] - 5s 2ms/step - loss: 0.5184 - accuracy: 0.7439
Epoch 2/10
267/267 [==============================] - 1s 2ms/step - loss: 0.3879 - accuracy: 0.8262
Epoch 3/10
267/267 [==============================] - 1s 2ms/step - loss: 0.3639 - accuracy: 0.8412
Epoch 4/10
267/267 [==============================] - 1s 2ms/step - loss: 0.3556 - accuracy: 0.8403
Epoch 5/10
267/267 [==============================] - 1s 2ms/step - loss: 0.3405 - accuracy: 0.8509
Epoch 6/10
267/267 [==============================] - 1s 2ms/step - loss: 0.3336 - accuracy: 0.8549
Epoch 7/10
267/267 [==============================] - 1s 2ms/step - loss: 0.3289 - accuracy: 0.8609
Epoch 8/10
267/267 [==============================] - 1s 2ms/step - loss: 0.3210 - accuracy: 0.8607
Epoch 9/10
267/267 [==============================] - 1s 2ms/step - loss: 0.3215 - accuracy: 0.8573
Epoch 10/10
267/267 [==============================] - 1s 2ms/step - loss: 0.3113 - accuracy: 0.8673

Text-Blob

In [ ]:
from textblob import TextBlob
y_pr=[]
for sentence in list(x_test_sent['sentence']):
  y_pr.append(TextBlob(sentence).sentiment.polarity)
y_pr
df=pd.DataFrame(y_pr,columns=['sent'])
df['sent']=pd.cut(df['sent'],bins=5,labels=[0,1,2,3,4])
accuracy_score(x_test['label'],df['sent'])

0.283710407239819

Vader Lexicon

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader=SentimentIntensityAnalyzer()
y_pr=[]
for sentence in list(x_test_sent['sentence']):
  y_pr.append(vader.polarity_scores(sentence)['compound'])
y_pr
df=pd.DataFrame(y_pr,columns=['sent'])
df['sent']=pd.cut(df['sent'],bins=2,labels=[0, 1])
accuracy_score(x_test['label'],df['sent'])

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


0.20950226244343892

ML CLASSIFIERS

In [ ]:
#lr.fit(np.array(sentence_embeddings),np.array(y_train_ind))
#lr.fit(np.array(sentence_embeddings),y_sst2_t)
#y_pred=lr.predict(np.array(sentence_embeddings_test))
#y_pred=lr.predict(np.array(sentence_embeddings_test))
#accuracy_score(np.array(y_test_ind),np.array(y_pred))
#accuracy_score(y_sst2_tes,np.array(y_pred))

#xgb.fit(np.array(sentence_embeddings),y_sst2_t)
#y_pred=xgb.predict(np.array(sentence_embeddings_test))
#accuracy_score(y_sst2_tes,np.array(y_pred))

#gb.fit(np.array(sentence_embeddings),y_sst2_t)
#y_pred=gb.predict(np.array(sentence_embeddings_test))
#accuracy_score(y_sst2_tes,np.array(y_pred))

#knn.fit(np.array(sentence_embeddings),np.array(y_train_ind))

rf.fit(np.array(sentence_embeddings),np.array(y_train_ind))
y_pred=rf.predict(np.array(sentence_embeddings_test))
accuracy_score(y_test_ind,y_pred)

#y_pred=rf.predict(np.array(sentence_embeddings_test))
#smx.fit(np.array(sentence_embeddings),np.array(y_train_ind))
#gnb.fit(np.array(sentence_embeddings),np.array(y_train_ind))

# l=[(r,z) for r,z in zip(y_test_ind,y_pred)]
# print(tuple(x)).[0:10]
# print(l)
# print(y_train_ind)

0.46289592760181

Converting SST-5 to SST-2

In [ ]:
def convert(n):
  if(n>2):
    n=1
  else:
    n=0
  return n 

In [ ]:
y_train_ind=np.array(y_train_ind)
y_test_ind=np.array(y_test_ind)
y_sst2_t=[]
y_sst2_tes=[]
for i in y_test_ind:
  y_sst2_tes.append(convert(i))
for i in y_train_ind:
  y_sst2_t.append(convert(i))
y_sst2_t=np.array(y_sst2_t)
y_sst2_tes=np.array(y_sst2_tes)
y_sst2_t
y_sst2_t.shape

(8544,)